In [ ]:
!pip install pyautogui
!pip install annoy
!pip install tensorflow_hub

In [ ]:
import annoy
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle
import pyautogui
import pyperclip
import pytesseract
import random
import tensorflow as tf
import tensorflow_hub as hub
import time

In [ ]:
def screenshot(bbox=None): #bbox format : left - top - width - height
    return pyautogui.screenshot(region=bbox)

FIGSIZE = (6,6)
def plot_img(image, figsize=FIGSIZE,cmap=None):
    plt.figure(figsize=figsize)
    plt.imshow(image,cmap=cmap)

In [ ]:
def start_game():
    pyautogui.click(242, 585)
    pyautogui.click(242, 585)

In [ ]:
triangle_area = (220,130,1,390)
threshold = 120
fn = lambda x : 255 if x> threshold else 0
resize = 1
screen = screenshot(triangle_area).convert("L")
plot_img(screen,cmap = 'gray')
array = np.array(screen)

In [ ]:
threshold = 120
fn = lambda x : 255 if x> threshold else 0
resize = 1
triangle_area = (220,130,1,390)

def take_screenshot():
    return np.array(screenshot(triangle_area).convert("L"))

def screenshot_text(array,line_index):
    
    text_area = (239,130 + line_index - 6, 240,21)
    screen = screenshot(triangle_area).convert("L")
    array = np.array(screen)
    text_screen = screenshot(text_area).convert("L")
    return np.array(text_screen)

In [ ]:
screen = take_screenshot()
line_index = np.where(screen >= (max(screen) - 8))[0][0]
text_screen = screenshot_text(screen,line_index)
plot_img(text_screen,cmap='gray')

In [ ]:
embedding_dimension = 64
index_filename = 'index2'
index = annoy.AnnoyIndex(embedding_dimension,metric = 'angular')
index.load(index_filename)
print('Annoy index is loaded.')
with open(index_filename + '.mapping', 'rb') as handle:
    mapping = pickle.load(handle)
print('Mapping file is loaded.')

In [ ]:
model_url = 'https://tfhub.dev/google/universal-sentence-encoder/4'
print("Loading the TF-Hub model...")
%time embed_fn = hub.load(model_url)
print("TF-Hub model is loaded.")

In [ ]:
def find_similar_items(embedding, num_matches=5):
    '''Finds similar items to a given embedding in the ANN index'''
    ids = index.get_nns_by_vector(embedding, num_matches, search_k=-1, include_distances=False)
    items = [mapping[i] for i in ids]
    return items

In [ ]:
random_projection_matrix = None
random_projection_matrix_filename = 'random_projection_matrix2'
if os.path.exists(random_projection_matrix_filename):
    print("Loading random projection matrix...")
    with open(random_projection_matrix_filename, 'rb') as handle:
        random_projection_matrix = pickle.load(handle)
    print('random projection matrix is loaded.')

def extract_embeddings(query):
    '''Generates the embedding for the query'''
    print(query, ' query')
    query_embedding =  embed_fn([query])[0].numpy()

    if random_projection_matrix is not None:
        query_embedding = random_projection_matrix.transform(query_embedding.reshape(1,-1)).reshape(-1,)
    return query_embedding

In [ ]:
query = 'cranberry'

print("Generating embedding for the query...")
%time query_embedding = extract_embeddings(query)

print("")
print("Finding relevant items in the index...")
%time items = find_similar_items(query_embedding, 10)

print("")
for item in items:
    print(item)
    
print(get_words(query,items))

In [ ]:
text = pytesseract.image_to_string(text_screen,config='--psm 10 --oem 3')
print(text)

In [ ]:
def begin_with(root,words) :
    return not root.startswith(words[:3])

def get_words(query,items): 
    return list(filter(lambda word : begin_with(query,word),items))


In [ ]:
screen = take_screenshot()
line_index = np.where(screen >= (max(screen) - 8))[0][0]
text_screen = screenshot_text(screen,line_index)
plot_img(text_screen,cmap='gray')

def run(verbose=True):
    go = False
    tim = time.time()
    line_index = -1
    while not go : 
        screen = take_screenshot()
        line_index_temp = np.where(screen >= (max(screen) - 8))[0][0]
        if line_index == line_index_temp : 
            break
        line_index = line_index_temp
        
    text_screen = screenshot_text(screen,line_index)
    if verbose : 
        plot_img(text_screen,cmap='gray')
    try : 
        text = pytesseract.image_to_string(text_screen,config='--psm 10 --oem 3')
        text = text.strip().lower()
    except IndexError : 
        return run()
    query_embedding = extract_embeddings(text)
    items = find_similar_items(query_embedding, 10)
    words = get_words(text,items)
    selected_words = words[random.randint(0,(len(words)-1))//2]
    print(items, words, selected_words)
    pyperclip.copy(selected_words)
    pyautogui.click(x=100, y=200)
    pyautogui.hotkey('command', 'v')
    pyautogui.press('enter')
    print(time.time() - tim, ' temps global')

In [ ]:
run(verbose=True)

In [ ]:
#Best score so far : 2870

while True: 
    run()